In [57]:
import json
import boto3
import os

In [16]:
#Reading configuration file
config_file = os.path.join(os.getcwd() , 'config.json')
with open(config_file) as c:
    credentials = json.load(c)
    
#Setting AcessKey and Secret Key to access s3 buckets.
ACESSKEY =  credentials['AWS']['AcessKey']
SECRETKEY = credentials['AWS']['AWSSecretKey']

In [187]:
# Creating a client to interact with s3 services
client = boto3.client('s3', aws_access_key_id = ACESSKEY, aws_secret_access_key = SECRETKEY)
response = client.list_buckets()

#Fetching the already created bucket name to upload the files into.
bucket_name = response['Buckets'][0]['Name']

#Local path where data lake is stored.
lake_directory = credentials['local_paths']['lake_path']


#Fetching all the subdirectories(modules) in datalake
sub_directories = [val[0] for val in os.walk(lake_directory)][1:]

In [189]:
for directories in sub_directories:
    for sub_files in os.walk(directories):
        s3_folder = directories.split('/')[3]
        for files in sub_files[2]:
            file_path = os.path.join(lake_directory + s3_folder + '/' + files)
            client.upload_file(file_path , bucket_name , (s3_folder+'/' + files))
    
    #Verifying Data Upload
    if verify_upload(s3_folder):
        print("Success !! Upload of "+ s3_folder + " dataset successfull.")
    else:
        print("Failure. Upload of " + s3_folder + " Failed.")

Success !! Upload of business dataset successfull.
Success !! Upload of business_attribute dataset successfull.
Success !! Upload of checkin dataset successfull.
Success !! Upload of opening_time dataset successfull.
Success !! Upload of photo dataset successfull.
Success !! Upload of review dataset successfull.
Success !! Upload of tip_files dataset successfull.
Success !! Upload of user dataset successfull.


In [181]:
def verify_upload(dataset_name):
    total_files_in_local = 0
    total_files_in_s3 = 0
    file_list_from_local = []
    file_list_from_s3 = []
    
    #Fetching number of files in local
    full_local_path = os.path.join(lake_directory , dataset_name)
    total_files_in_local = [len(val[2]) for val in os.walk(full_local_path)]
    
    
    #Fetching number of files in s3
    request = client.list_objects(Bucket = bucket_name , Prefix = dataset_name)
    total_files_in_s3 = len(request['Contents'])
    
    #Validating File names
    file_list_from_local = [val[2] for val in os.walk(full_local_path)][0]
    
    file_list_from_s3 = [content['Key'].split('/')[-1] for content in request['Contents']]
    
    if set(file_list_from_local).difference(set(file_list_from_s3)) == set([]):
        return True
    else:
        return False

In [182]:
verify_upload('business_attribute')

True